# Imports and functions

In [1]:
import os
import tensorflow as tf
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator
import numpy as np
import matplotlib.pyplot as plt
%matplotlib notebook

from collections import OrderedDict, namedtuple

def extract_tensor_from_summary(summary_value, dt=None):
    """
    De-serializes data from tensorflow event
    :param summary_value: event.summary.value object created by summary writer
    :return deci_tensor:  decimal numpy array
    """
    _dt = dt if dt else tf.as_dtype(summary_value.tensor.dtype).as_numpy_dtype()
    shape = tuple([i.size for i in summary_value.tensor.tensor_shape.dim])
    print(shape)
    binary_tensor = summary_value.tensor.tensor_content
#     print(np.fromstring(binary_tensor, dtype=dt))
    deci_tensor = np.fromstring(binary_tensor, dtype=dt).reshape(shape)
    return deci_tensor


def retrieve_model_params(path_to_event_file, layer_name, param_name):
    """
    Retrieves parameter values from a tensorflow event file and structures it into an ordered dict
    :param path_to_event_file: absolute path to event file
    :param layer_name: string name of the layer
    :param params_summary: stirng name of the parameter to retrieve (e.g. 'weights')
    :param data_type (optional): data type of output numpy arrays (if ommited, original data type converted from 
        tensorflow DType to numpy dtype will be used
    :return: params_dict: an ordered dict where keys are global steps (of type int) and values are numpy arrays
    """
    params_dict = OrderedDict()
    lookup = (layer_name, param_name)
    for event in tf.train.summary_iterator(path_to_event_file):
        for val in event.summary.value:
            if all([item in val.tag for item in lookup]):
                params_dict[event.step] = tf.contrib.util.make_ndarray(val.tensor)
    return params_dict


def retrieve_model_data(path_to_event_file, layer_name, tensor_name):
    data_dict = OrderedDict()
    SummaryData = namedtuple('SummaryData', ['labels','inputs','targets','data'])
    lookup = '/'.join([layer_name,tensor_name])
    for i, event in enumerate(tf.train.summary_iterator(event_file)):
        data_dict.setdefault(event.step, SummaryData([],[],[],[]))
        for val in event.summary.value:
            if 'pattern_labels' in val.tag:
                data_dict[event.step].labels.append(val.tensor.string_val[0].decode('utf-8'))
            if 'input_patterns' in val.tag:
                data_dict[event.step].inputs.append(tf.contrib.util.make_ndarray(val.tensor))
            if 'target_patterns' in val.tag:
                data_dict[event.step].targets.append(tf.contrib.util.make_ndarray(val.tensor))
            if lookup in val.tag:
                data_dict[event.step].data.append(tf.contrib.util.make_ndarray(val.tensor))

    return data_dict


def retrieve_loss(path_to_event_file):
    eacc = get_event_accumulator(path_to_event_file)
    loss_log = np.stack([np.asarray([scalar.step, scalar.value]) for scalar in eacc.Scalars('train/error_summary')])
    return loss_log#    


def display_model_data(data_dict):
    for step, data in data_dict.items():
        print('> Epoch',step)
        print('  labels:')
        for x in data.labels: print(x)
        print('  inputs:')
        for x in data.inputs: print(x)
        print('  targets:')
        for x in data.targets: print(x)
        print('  data:')
        for x in data.data: print(x)

# Event directory path

In [3]:
# PATH VARS
logdir = '/Users/alexten/Projects/pdpyflow/xor/train/log_000'
event_file = os.path.join(logdir, os.listdir(logdir)[0])

# List of all (unique) summaries

In [ ]:
tag_list = []
for event in tf.train.summary_iterator(event_file):
    for val in event.summary.value:
        tag_list.append(val.tag)
for tag in sorted(set(tag_list)):
    print(tag)

# Get summary by tag 

In [ ]:
lookup = ['output_layer','gradient-weights']
for event in tf.train.summary_iterator(event_file):
    for val in event.summary.value:
        if all([item in val.tag for item in lookup]):
            print(event.step)
            print(tf.contrib.util.make_ndarray(val.tensor))
#             print(val)

# Test functions

In [5]:
hidden_weights = retrieve_model_params(event_file, 'hidden_layer', 'weights')
hidden_bias = retrieve_model_params(event_file, 'hidden_layer', 'bias')
hidden_acts = retrieve_model_data(event_file, 'hidden_layer', 'gradient_weights')
# print(hidden_acts[0].labels)
display_model_data(hidden_acts)

> Epoch 0
  labels:
p00
p01
p10
p11
  inputs:
[[ 0.  0.]]
[[ 0.  1.]]
[[ 1.  0.]]
[[ 1.  1.]]
  targets:
[[ 0.]]
[[ 1.]]
[[ 1.]]
[[ 0.]]
  data:
[[[ 0.  0.]
  [ 0.  0.]]]
[[[ 0.          0.        ]
  [-0.01239182 -0.00362648]]]
[[[-0.01243565 -0.0035806 ]
  [ 0.          0.        ]]]
[[[ 0.01815801  0.00572924]
  [ 0.01815801  0.00572924]]]
> Epoch 30
  labels:
p00
p01
p10
p11
  inputs:
[[ 0.  0.]]
[[ 0.  1.]]
[[ 1.  0.]]
[[ 1.  1.]]
  targets:
[[ 0.]]
[[ 1.]]
[[ 1.]]
[[ 0.]]
  data:
[[[ 0.  0.]
  [ 0.  0.]]]
[[[ 0.          0.        ]
  [-0.00761698  0.00090695]]]
[[[-0.00761462  0.00088839]
  [ 0.          0.        ]]]
[[[ 0.00815999 -0.00096241]
  [ 0.00815999 -0.00096241]]]
> Epoch 60
  labels:
p00
p01
p10
p11
  inputs:
[[ 0.  0.]]
[[ 0.  1.]]
[[ 1.  0.]]
[[ 1.  1.]]
  targets:
[[ 0.]]
[[ 1.]]
[[ 1.]]
[[ 0.]]
  data:
[[[ 0.  0.]
  [ 0.  0.]]]
[[[ 0.          0.        ]
  [-0.00319089  0.00200099]]]
[[[-0.00318931  0.00196098]
  [ 0.          0.        ]]]
[[[ 0.00314384 -0.001